In [1]:
using BenchmarkTools
using Distances
using MLDatasets
using NNDescent
using NNDescent: brute_knn, brute_search, NNTuple

In [2]:
recall(nn, true_nn) = sum(_recall(nn[:,i], true_nn[:,i]) for i in 1:size(nn,2))/size(nn,2)
_recall(π, πₜ) = length(intersect(π, πₜ))/length(πₜ)

_recall (generic function with 1 method)

In [3]:
train_x, train_y = FashionMNIST.traindata()
test_x, test_y = FashionMNIST.testdata()
train_x = reshape(train_x, size(train_x)[1]*size(train_x)[2], size(train_x)[3])
test_x = reshape(test_x, size(test_x)[1]*size(test_x)[2], size(test_x)[3])

data = [convert.(Float32, train_x[:,i]) for i = 1:size(train_x, 2)]
data = data[1:5000]
queries = [convert.(Float32, test_x[:,i]) for i = 1:size(test_x, 2)]
queries = queries[1:500];

In [4]:
knn_graph = DescentGraph(data, 10)
nn = getindex.(knn_graph.graph, 1);

In [5]:
brute_graph = brute_knn(data, Euclidean(), 10)
true_nn = getindex.(brute_graph, 1);

In [6]:
recall(nn, true_nn)

0.9964999999999998

In [7]:
true_idx, true_dist = brute_search(data, queries, 10, Euclidean());

In [8]:
idx, dist = search(knn_graph, queries, 10, 70);

In [9]:
recall(idx, true_idx)

0.8865999999999992

In [10]:
q_per_sec = length(queries)/(@belapsed search(knn_graph, queries, 10, 70))

330.94124478737336

In [11]:
@benchmark DescentGraph(data, 10)

BenchmarkTools.Trial: 
  memory estimate:  505.40 MiB
  allocs estimate:  10388471
  --------------
  minimum time:     989.240 ms (12.54% GC)
  median time:      1.029 s (14.36% GC)
  mean time:        1.034 s (15.30% GC)
  maximum time:     1.101 s (18.45% GC)
  --------------
  samples:          5
  evals/sample:     1

In [12]:
@benchmark search(knn_graph, queries, 10, 70)

BenchmarkTools.Trial: 
  memory estimate:  691.46 MiB
  allocs estimate:  25057017
  --------------
  minimum time:     1.527 s (10.49% GC)
  median time:      1.605 s (14.83% GC)
  mean time:        1.618 s (14.63% GC)
  maximum time:     1.734 s (17.92% GC)
  --------------
  samples:          4
  evals/sample:     1